In [1]:
import django_initializer
from telemetry.models import Game, Driver, Car, Track, SessionType, Lap, FastLap
from telemetry.influx import Influx
from telemetry.analyzer import Analyzer
import plotly.express as px
import plotly.io as pio

# pio.renderers.default = "svg"  # comment this line to use interactive plots
import plotly.graph_objects as go
import pandas as pd

%load_ext autoreload
%autoreload 2

analyzer = Analyzer()
influx = Influx()

2023-05-04 19:24:16,208 DEBUG Influx: Connected to http://telemetry.b4mad.racing/


In [10]:
measurement = "fast_laps"
bucket = "fast_laps"
# measurement = "laps_cc"
# bucket = "racing"
lap = Lap.objects.get(id=2084)
session_df = influx.telemetry_for_laps([lap], measurement=measurement, bucket=bucket)[0]

2023-05-04 19:26:48,128 INFO Processing iRacing spielberg gp : session 1680444939 : lap.id 2084 : length 4279 : time 89.8657


In [11]:
session_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5165 entries, 0 to 5227
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype                    
---  ------               --------------  -----                    
 0   result               5165 non-null   object                   
 1   table                5165 non-null   int64                    
 2   _start               5165 non-null   datetime64[ns, tzlocal()]
 3   _stop                5165 non-null   datetime64[ns, tzlocal()]
 4   _time                5165 non-null   datetime64[ns, tzlocal()]
 5   CarModel             5165 non-null   object                   
 6   CurrentLap           5165 non-null   object                   
 7   GameName             5165 non-null   object                   
 8   SessionId            5165 non-null   object                   
 9   SessionTypeName      5165 non-null   object                   
 10  TrackCode            5165 non-null   object                   
 11  _measurem

In [12]:
df = session_df.copy()

In [13]:
def lap_fig_with_replay_v2(df):
    fig = go.Figure()

    fig.add_scatter(
        x=df["_time"],
        y=df["DistanceRoundTrack"],
        mode="markers",
        marker=dict(size=1),
        name="SpeedMsNormalized",
    )

    bool_columns = ["CurrentLapIsValid", "PreviousLapWasValid"]
    for col in bool_columns:
        col_changes = df[df[col].shift(1) != df[col]].copy()
        col_changes["color"] = col_changes[col].apply(lambda x: "green" if x else "red")

        for _, row in col_changes.iterrows():
            fig.add_shape(
                type="line",
                x0=row["_time"],
                x1=row["_time"],
                y0=0,
                y1=row["DistanceRoundTrack"],
                yref="y",
                xref="x",
                line=dict(color=row["color"], width=1),
            )

    fig.add_scatter(
        x=df["_time"],
        y=df["CurrentLap"],
        mode="lines+markers",
        marker=dict(size=3),
        name="CurrentLap",
    )

    fig.add_scatter(
        x=df["_time"],
        y=df["LapTimePrevious"],
        mode="lines+markers",
        marker=dict(size=3),
        name="LapTimePrevious",
    )

    return fig


lap_fig_with_replay_v2(df).show()

Normalize the float columns, so that the value is between 0 and 1. Also, just plot the LapTimePrevious as a large dot, if the value changes.

In [14]:
def normalize(series):
    return (series - series.min()) / (series.max() - series.min())


def lap_fig_with_replay_v3(df):
    fig = go.Figure()

    df["DistanceRoundTrack"] = normalize(df["DistanceRoundTrack"])

    fig.add_scatter(
        x=df["_time"],
        y=df["DistanceRoundTrack"],
        mode="markers",
        marker=dict(size=1),
    )

    bool_columns = ["CurrentLapIsValid", "PreviousLapWasValid"]
    for col in bool_columns:
        col_changes = df[df[col].shift(1) != df[col]].copy()
        col_changes["color"] = col_changes[col].apply(lambda x: "green" if x else "red")

        for _, row in col_changes.iterrows():
            fig.add_shape(
                type="line",
                x0=row["_time"],
                x1=row["_time"],
                y0=0,
                y1=row["DistanceRoundTrack"],
                yref="y",
                xref="x",
                line=dict(color=row["color"], width=1),
                name=col,
            )

    fig.add_scatter(
        x=df["_time"],
        y=df["CurrentLap"],
        mode="markers",
        marker=dict(size=3),
        name="CurrentLap",
    )

    df["LapTimePrevious"] = normalize(df["LapTimePrevious"])
    lap_time_changes = df[
        df["LapTimePrevious"].shift(1) != df["LapTimePrevious"]
    ].copy()
    fig.add_scatter(
        x=lap_time_changes["_time"],
        y=lap_time_changes["LapTimePrevious"],
        mode="markers",
        marker=dict(size=8),
        name="LapTimePrevious",
    )

    return fig


lap_fig_with_replay_v3(df).show()